# CytoTable (convert) and Pycytominer (SingleCells.merge_single_cells) Performance Comparisons

This notebook explores CytoTable (convert) and Pycytominer (SingleCells.merge_single_cells) usage with datasets of varying size to help describe performance impacts.

In [1]:
import itertools
import pathlib
import re
import subprocess

import pandas as pd
import plotly.express as px
import plotly.io as pio
from IPython.display import Image
from utilities import (
    get_memory_peak_and_time_duration,
    get_parsl_peak_memory,
    get_system_info,
)

# set plotly default theme
pio.templates.default = "simple_white"
# monitoring database for parsl multiprocessing work
db_file = "runinfo/monitoring.db"

In [2]:
# show the system information
_ = get_system_info(show_output=True)


System Information:
Operating System: Linux
Machine Type: x86_64
Processor: x86_64
CPU Cores (Logical): 16
CPU Cores (Physical): 8
Total RAM (GB): 62.68
Python Version: 3.11.9


In [3]:
# observe the virtual env for dependency inheritance with memray
# from subprocedure calls
"/".join(
    subprocess.run(
        [
            "which",
            "python",
        ],
        capture_output=True,
        check=True,
    )
    # decode bytestring as utf-8
    .stdout.decode("utf-8")
    # remove personal file structure
    .split("/")[6:]
    # replace final newline
).replace("\n", "")

'cytotable-benchmarks-2FApAEcz-py3.11/bin/python'

In [4]:
# target file or table names
image_dir = "images"
examples_dir = "examples"
join_read_time_image = (
    f"{image_dir}/cytotable-and-pycytominer-comparisons-join-completion-time-sqlite.png"
)
join_mem_size_image = (
    f"{image_dir}/cytotable-and-pycytominer-comparisons-join-memory-size-sqlite.png"
)
example_files_list = [
    f"{examples_dir}/cytotable_convert_nf1_multiprocess_sqlite.py",
    f"{examples_dir}/cytotable_convert_nf1_multithread_sqlite.py",
    f"{examples_dir}/pycytominer_merge_nf1_sqlite.py",
]
example_data_list = [
    f"{examples_dir}/data/all_cellprofiler.sqlite",
    f"{examples_dir}/data/all_cellprofiler-x2.sqlite",
    f"{examples_dir}/data/all_cellprofiler-x4.sqlite",
    f"{examples_dir}/data/all_cellprofiler-x8.sqlite",
    f"{examples_dir}/data/all_cellprofiler-x16.sqlite",
    f"{examples_dir}/data/all_cellprofiler-x32.sqlite",
    f"{examples_dir}/data/all_cellprofiler-x64.sqlite",
    f"{examples_dir}/data/all_cellprofiler-x128.sqlite",
    f"{examples_dir}/data/all_cellprofiler-x256.sqlite",
    f"{examples_dir}/data/all_cellprofiler-x512.sqlite",
]

# format for memray time strings
tformat = "%Y-%m-%d %H:%M:%S.%f%z"

In [5]:
# Define the Parquet file path
results_parquet_file = "cytotable_pycytominer_results.parquet"

# Load existing results if available
if pathlib.Path(results_parquet_file).exists():
    df_results = pd.read_parquet(results_parquet_file)
    results = df_results.to_dict(orient="records")
else:
    results = []

In [6]:
# Number of iterations for each combination
num_iterations = 6

# Loop through each combination of example file and data file
for example_file, example_data in itertools.product(
    example_files_list, example_data_list
):
    for iteration in range(num_iterations):
        print(f"Starting {example_file} with {example_data}, iteration {iteration}.")
        # Skip if this combination and iteration are already processed
        if any(
            result["file_input"] == example_file
            and result["data_input"] == example_data
            and result["iteration"] == iteration
            for result in results
        ):
            print(
                f"Skipping already processed: {example_file} with {example_data}, iteration {iteration}"
            )
            continue

        try:
            # gather memory peak and time duration
            memory_peak, time_duration = get_memory_peak_and_time_duration(
                cmd=[
                    "python",
                    example_file,
                    example_data,
                ],
                polling_pause_seconds=0.1,
                # if we have a multiprocessed parsl process skip memory
                # (we will check this via parsl monitoring).
                skip_memory_check=("multiprocess" in example_file),
            )

            # Append the result
            results.append(
                {
                    "file_input": example_file.replace(f"{examples_dir}/", ""),
                    "data_input": example_data,
                    "iteration": iteration,
                    "time_duration (secs)": time_duration,
                    "peak_memory (bytes)": (
                        memory_peak
                        # if we have a multiprocessed parsl result we must
                        # gather the peak memory using parsl's monitoring
                        # database.
                        if "multiprocess" not in example_file
                        else get_parsl_peak_memory(db_file=db_file)
                    ),
                }
            )

            # Save intermediate results to Parquet
            df_results = pd.DataFrame(results)
            df_results.to_parquet(results_parquet_file, index=False)

        except Exception as e:
            print(
                f"Error processing {example_file} with {example_data}, iteration {iteration}: {e}"
            )

        finally:
            # remove monitoring database if present from parsl processing
            if pathlib.Path(db_file).is_file():
                pathlib.Path(db_file).unlink()
            print(
                f"Finished {example_file} with {example_data}, iteration {iteration}."
            )


# Final save to Parquet
df_results = pd.DataFrame(results)
df_results.to_parquet(results_parquet_file, index=False)

print(f"Processing complete. Results saved to '{results_parquet_file}'.")

Starting examples/cytotable_convert_nf1_multiprocess_sqlite.py with examples/data/all_cellprofiler.sqlite, iteration 0.


Finished examples/cytotable_convert_nf1_multiprocess_sqlite.py with examples/data/all_cellprofiler.sqlite, iteration 0.
Starting examples/cytotable_convert_nf1_multiprocess_sqlite.py with examples/data/all_cellprofiler.sqlite, iteration 1.


Finished examples/cytotable_convert_nf1_multiprocess_sqlite.py with examples/data/all_cellprofiler.sqlite, iteration 1.
Starting examples/cytotable_convert_nf1_multiprocess_sqlite.py with examples/data/all_cellprofiler.sqlite, iteration 2.


Finished examples/cytotable_convert_nf1_multiprocess_sqlite.py with examples/data/all_cellprofiler.sqlite, iteration 2.
Starting examples/cytotable_convert_nf1_multiprocess_sqlite.py with examples/data/all_cellprofiler.sqlite, iteration 3.


Finished examples/cytotable_convert_nf1_multiprocess_sqlite.py with examples/data/all_cellprofiler.sqlite, iteration 3.
Starting examples/cytotable_convert_nf1_multiprocess_sqlite.py with examples/data/all_cellprofiler.sqlite, iteration 4.


...

Finished examples/pycytominer_merge_nf1_sqlite.py with examples/data/all_cellprofiler-x512.sqlite, iteration 3.
Starting examples/pycytominer_merge_nf1_sqlite.py with examples/data/all_cellprofiler-x512.sqlite, iteration 4.


/home/davebunten/.cache/pypoetry/virtualenvs/cytotable-benchmarks-2FApAEcz-py3.11/lib/python3.11/site-packages/pycytominer/cyto_utils/util.py:59: UserWarning: Non-canonical compartment detected: Per_Cells, Per_Cytoplasm, Per_Nuclei
  warnings.warn(warn_str)


Finished examples/pycytominer_merge_nf1_sqlite.py with examples/data/all_cellprofiler-x512.sqlite, iteration 4.
Starting examples/pycytominer_merge_nf1_sqlite.py with examples/data/all_cellprofiler-x512.sqlite, iteration 5.


/home/davebunten/.cache/pypoetry/virtualenvs/cytotable-benchmarks-2FApAEcz-py3.11/lib/python3.11/site-packages/pycytominer/cyto_utils/util.py:59: UserWarning: Non-canonical compartment detected: Per_Cells, Per_Cytoplasm, Per_Nuclei
  warnings.warn(warn_str)


Finished examples/pycytominer_merge_nf1_sqlite.py with examples/data/all_cellprofiler-x512.sqlite, iteration 5.
Processing complete. Results saved to 'cytotable_pycytominer_results.parquet'.


In [7]:
# add columns for data understandability in plots


def get_file_size_mb(file_path):
    """
    Gather filesize given a file_path
    """
    try:
        return pathlib.Path(file_path).stat().st_size / 1024 / 1024
    except FileNotFoundError:
        return None


# memory usage in MB
df_results["peak_memory (GB)"] = df_results["peak_memory (bytes)"] / 1024 / 1024 / 1024

# data input size additions for greater context
df_results["data_input_size_mb"] = df_results["data_input"].apply(get_file_size_mb)
df_results["data_input_with_size"] = (
    df_results["data_input"]
    + " ("
    + round(df_results["data_input_size_mb"]).astype("int64").astype("str")
    + " MB)"
)

# rename data input to simplify
df_results["data_input_renamed"] = (
    df_results["data_input_with_size"]
    .str.replace(f"{examples_dir}/data/", "")
    .str.replace("all_cellprofiler", "input")
)
df_results

,file_input,data_input,iteration,time_duration (secs),peak_memory (bytes),peak_memory (GB),data_input_size_mb,data_input_with_size,data_input_renamed
0,cytotable_convert_nf1_multiprocess_sqlite.py,examples/data/all_cellprofiler.sqlite,0,21.384496,3.631473e+08,0.338207,3.843750,examples/data/all_cellprofiler.sqlite (4 MB),input.sqlite (4 MB)
1,cytotable_convert_nf1_multiprocess_sqlite.py,examples/data/all_cellprofiler.sqlite,1,21.018959,3.596739e+08,0.334972,3.843750,examples/data/all_cellprofiler.sqlite (4 MB),input.sqlite (4 MB)
2,cytotable_convert_nf1_multiprocess_sqlite.py,examples/data/all_cellprofiler.sqlite,2,21.405169,3.521085e+08,0.327927,3.843750,examples/data/all_cellprofiler.sqlite (4 MB),input.sqlite (4 MB)
3,cytotable_convert_nf1_multiprocess_sqlite.py,examples/data/all_cellprofiler.sqlite,3,21.244814,3.731005e+08,0.347477,3.843750,examples/data/all_cellprofiler.sqlite (4 MB),input.sqlite (4 MB)
4,cytotable_convert_nf1_multiprocess_sqlite.py,examples/data/all_cellprofiler.sqlite,4,21.363728,3.641754e+08,0.339165,3.843750,examples/data/all_cellprofiler.sqlite (4 MB),input.sqlite (4 MB)
...,...,...,...,...,...,...,...,...,...
175,pycytominer_merge_nf1_sqlite.py,examples/data/all_cellprofiler-x512.sqlite,1,139.029644,8.660050e+09,8.065300,1909.335938,examples/data/all_cellprofiler-x512.sqlite (19...,input-x512.sqlite (1909 MB)
176,pycytominer_merge_nf1_sqlite.py,examples/data/all_cellprofiler-x512.sqlite,2,137.958207,8.659104e+09,8.064419,1909.335938,examples/data/all_cellprofiler-x512.sqlite (19...,input-x512.sqlite (1909 MB)
177,pycytominer_merge_nf1_sqlite.py,examples/data/all_cellprofiler-x512.sqlite,3,138.411487,8.659165e+09,8.064476,1909.335938,examples/data/all_cellprofiler-x512.sqlite (19...,input-x512.sqlite (1909 MB)
178,pycytominer_merge_nf1_sqlite.py,examples/data/all_cellprofiler-x512.sqlite,4,139.019377,8.659087e+09,8.064404,1909.335938,examples/data/all_cellprofiler-x512.sqlite (19...,input-x512.sqlite (1909 MB)


In [8]:
# build cols for split reference in the plot
df_results["cytotable_time_duration (multiprocess) (secs)"] = df_results[
    df_results["file_input"] == "cytotable_convert_nf1_multiprocess_sqlite.py"
]["time_duration (secs)"]
df_results["cytotable_peak_memory (multiprocess) (GB)"] = df_results[
    df_results["file_input"] == "cytotable_convert_nf1_multiprocess_sqlite.py"
]["peak_memory (GB)"]
df_results["cytotable_time_duration (multithread) (secs)"] = df_results[
    df_results["file_input"] == "cytotable_convert_nf1_multithread_sqlite.py"
]["time_duration (secs)"]
df_results["cytotable_peak_memory (multithread) (GB)"] = df_results[
    df_results["file_input"] == "cytotable_convert_nf1_multithread_sqlite.py"
]["peak_memory (GB)"]
df_results["pycytominer_time_duration (secs)"] = df_results[
    df_results["file_input"] == "pycytominer_merge_nf1_sqlite.py"
]["time_duration (secs)"]
df_results["pycytominer_peak_memory (GB)"] = df_results[
    df_results["file_input"] == "pycytominer_merge_nf1_sqlite.py"
]["peak_memory (GB)"]
df_results = (
    df_results.apply(lambda x: pd.Series(x.dropna().values))
    .drop(["file_input", "time_duration (secs)", "peak_memory (GB)"], axis=1)
    .dropna()
)
df_results

,data_input,iteration,peak_memory (bytes),data_input_size_mb,data_input_with_size,data_input_renamed,cytotable_time_duration (multiprocess) (secs),cytotable_peak_memory (multiprocess) (GB),cytotable_time_duration (multithread) (secs),cytotable_peak_memory (multithread) (GB),pycytominer_time_duration (secs),pycytominer_peak_memory (GB)
0,examples/data/all_cellprofiler.sqlite,0,3.631473e+08,3.843750,examples/data/all_cellprofiler.sqlite (4 MB),input.sqlite (4 MB),21.384496,0.338207,3.382802,0.311058,1.776670,0.207531
1,examples/data/all_cellprofiler.sqlite,1,3.596739e+08,3.843750,examples/data/all_cellprofiler.sqlite (4 MB),input.sqlite (4 MB),21.018959,0.334972,3.359946,0.333794,1.768085,0.207428
2,examples/data/all_cellprofiler.sqlite,2,3.521085e+08,3.843750,examples/data/all_cellprofiler.sqlite (4 MB),input.sqlite (4 MB),21.405169,0.327927,3.406309,0.317390,1.775051,0.207069
3,examples/data/all_cellprofiler.sqlite,3,3.731005e+08,3.843750,examples/data/all_cellprofiler.sqlite (4 MB),input.sqlite (4 MB),21.244814,0.347477,3.450309,0.318626,1.773164,0.206005
4,examples/data/all_cellprofiler.sqlite,4,3.641754e+08,3.843750,examples/data/all_cellprofiler.sqlite (4 MB),input.sqlite (4 MB),21.363728,0.339165,3.344983,0.301388,1.778455,0.207150
5,examples/data/all_cellprofiler.sqlite,5,3.526410e+08,3.843750,examples/data/all_cellprofiler.sqlite (4 MB),input.sqlite (4 MB),21.729096,0.328423,3.339835,0.302345,1.743157,0.207054
6,examples/data/all_cellprofiler-x2.sqlite,0,3.561923e+08,7.578125,examples/data/all_cellprofiler-x2.sqlite (8 MB),input-x2.sqlite (8 MB),21.973524,0.331730,3.400347,0.301991,1.958470,0.221584
7,examples/data/all_cellprofiler-x2.sqlite,1,3.561636e+08,7.578125,examples/data/all_cellprofiler-x2.sqlite (8 MB),input-x2.sqlite (8 MB),21.370225,0.331703,3.443927,0.320011,1.955712,0.221718
8,examples/data/all_cellprofiler-x2.sqlite,2,3.756278e+08,7.578125,examples/data/all_cellprofiler-x2.sqlite (8 MB),input-x2.sqlite (8 MB),21.611075,0.349831,3.447607,0.318329,1.979242,0.221439
9,examples/data/all_cellprofiler-x2.sqlite,3,3.925975e+08,7.578125,examples/data/all_cellprofiler-x2.sqlite (8 MB),input-x2.sqlite (8 MB),21.235337,0.365635,3.452294,0.346390,2.037822,0.215111


In [9]:
# Group by data_input_renamed and calculate mean, min, and max
aggregated_results = df_results.groupby("data_input_renamed").agg(
    {
        "cytotable_time_duration (multiprocess) (secs)": ["mean", "min", "max"],
        "cytotable_time_duration (multithread) (secs)": ["mean", "min", "max"],
        "pycytominer_time_duration (secs)": ["mean", "min", "max"],
        "cytotable_peak_memory (multiprocess) (GB)": ["mean", "min", "max"],
        "cytotable_peak_memory (multithread) (GB)": ["mean", "min", "max"],
        "pycytominer_peak_memory (GB)": ["mean", "min", "max"],
    }
)
# Flatten the multi-level columns
aggregated_results.columns = [
    f"{col[0]} ({col[1]})" for col in aggregated_results.columns
]
aggregated_results.reset_index(inplace=True)


# Helper function to extract numeric value or None
def sort_key(s):
    match = re.search(r"\d+", s)
    if match:
        return (1, int(match.group()))  # numeric items: (1, number)
    else:
        return (0, s.lower())  # non-numeric items: (0, alphabetical)


# Sort using the custom key
aggregated_results = aggregated_results.sort_values(
    by="data_input_renamed", key=lambda col: col.map(sort_key)
)
aggregated_results

,data_input_renamed,cytotable_time_duration (multiprocess) (secs) (mean),cytotable_time_duration (multiprocess) (secs) (min),cytotable_time_duration (multiprocess) (secs) (max),cytotable_time_duration (multithread) (secs) (mean),cytotable_time_duration (multithread) (secs) (min),cytotable_time_duration (multithread) (secs) (max),pycytominer_time_duration (secs) (mean),pycytominer_time_duration (secs) (min),pycytominer_time_duration (secs) (max),cytotable_peak_memory (multiprocess) (GB) (mean),cytotable_peak_memory (multiprocess) (GB) (min),cytotable_peak_memory (multiprocess) (GB) (max),cytotable_peak_memory (multithread) (GB) (mean),cytotable_peak_memory (multithread) (GB) (min),cytotable_peak_memory (multithread) (GB) (max),pycytominer_peak_memory (GB) (mean),pycytominer_peak_memory (GB) (min),pycytominer_peak_memory (GB) (max)
2,input-x2.sqlite (8 MB),21.482275,21.235337,21.973524,3.443471,3.400347,3.500978,1.982614,1.955712,2.037822,0.340949,0.331703,0.365635,0.329437,0.301991,0.350273,0.221341,0.215111,0.226254
5,input-x4.sqlite (15 MB),21.367945,21.169588,21.693600,3.637143,3.587915,3.670504,2.346181,2.331016,2.367945,0.356510,0.351368,0.371887,0.381286,0.351509,0.414268,0.251218,0.250782,0.251541
9,input.sqlite (4 MB),21.357710,21.018959,21.729096,3.380697,3.339835,3.450309,1.769097,1.743157,1.778455,0.336028,0.327927,0.347477,0.314100,0.301388,0.333794,0.207040,0.206005,0.207531
8,input-x8.sqlite (30 MB),21.676985,21.540652,21.770709,4.040147,3.985098,4.078631,3.185581,3.146029,3.263625,0.429759,0.408226,0.483082,0.443315,0.409195,0.470730,0.309583,0.304459,0.315948
1,input-x16.sqlite (60 MB),22.113636,21.928231,22.404097,4.456335,4.346780,4.700246,4.984013,4.942839,5.044000,0.516483,0.466084,0.560818,0.561344,0.532894,0.584862,0.436341,0.434032,0.438606
4,input-x32.sqlite (119 MB),22.984812,22.647534,23.631211,5.366532,5.205241,5.560186,8.516205,8.265502,8.703588,0.670774,0.611977,0.723980,0.686238,0.605682,0.751198,0.684088,0.683960,0.684231
7,input-x64.sqlite (239 MB),24.656177,24.081396,24.936935,7.268257,6.922514,7.560812,16.226874,15.793225,16.534453,0.934532,0.769905,1.067928,1.017760,0.919926,1.154560,1.173052,1.172638,1.173416
0,input-x128.sqlite (477 MB),28.494632,28.198151,28.863461,10.957092,10.596211,11.667468,33.751474,33.527879,34.400265,1.338342,1.261059,1.390713,1.680670,1.608109,1.791695,2.156260,2.156013,2.156483
3,input-x256.sqlite (955 MB),34.579539,34.270991,34.833079,17.813752,16.877595,19.243667,66.865408,66.421230,67.363990,1.846376,1.794670,1.903576,2.656963,2.558720,2.700455,4.124838,4.124538,4.125107
6,input-x512.sqlite (1909 MB),49.147961,47.990966,49.685872,34.111594,33.528387,34.684057,138.606760,137.958207,139.114694,2.838564,2.769634,3.045765,4.170724,3.942654,4.279091,8.064796,8.064404,8.065300


In [10]:
# Time plot with min and max errors
fig = px.line(
    aggregated_results,
    x="data_input_renamed",
    y=[
        "cytotable_time_duration (multiprocess) (secs) (mean)",
        "cytotable_time_duration (multithread) (secs) (mean)",
        "pycytominer_time_duration (secs) (mean)",
    ],
    title="CytoTable and Pycytominer SQLite Processing Time with Min/Max Errors",
    labels={"data_input_renamed": "Input File", "value": "Seconds"},
    height=500,
    width=900,
    symbol_sequence=["diamond"],
    color_discrete_sequence=[
        px.colors.qualitative.Vivid[6],
        px.colors.qualitative.Vivid[7],
        px.colors.qualitative.Vivid[4],
    ],
)

# Add error bars for each trace
for i, col in enumerate(
    [
        "cytotable_time_duration (multiprocess) (secs)",
        "cytotable_time_duration (multithread) (secs)",
        "pycytominer_time_duration (secs)",
    ]
):
    fig.data[i].update(
        error_y=dict(
            array=(
                aggregated_results[f"{col} (max)"] - aggregated_results[f"{col} (mean)"]
            ),
            arrayminus=(
                aggregated_results[f"{col} (mean)"] - aggregated_results[f"{col} (min)"]
            ),
        )
    )

# Rename the lines for the legend
newnames = {
    "cytotable_time_duration (multiprocess) (secs) (mean)": "CytoTable (multiprocess)",
    "cytotable_time_duration (multithread) (secs) (mean)": "CytoTable (multithread)",
    "pycytominer_time_duration (secs) (mean)": "Pycytominer",
}
fig.for_each_trace(
    lambda t: t.update(
        name=newnames[t.name],
        legendgroup=newnames[t.name],
        hovertemplate=t.hovertemplate.replace(t.name, newnames[t.name]),
    )
)

# Update the legend and layout
fig.update_layout(
    legend_title_text="",
    legend=dict(x=0.01, y=0.98, bgcolor="rgba(255,255,255,0.8)"),
    font=dict(size=16),  # Global font size
)
fig.update_traces(mode="lines+markers")
fig.update_layout(yaxis=dict(rangemode="tozero", autorange=True))

# Save and display the plot
fig.write_image(join_read_time_image)
fig.write_image(join_read_time_image.replace(".png", ".svg"))
Image(url=join_read_time_image.replace(".png", ".svg"))

In [11]:
# Memory plot with min and max errors
fig = px.line(
    aggregated_results,
    x="data_input_renamed",
    y=[
        "cytotable_peak_memory (multiprocess) (GB) (mean)",
        "cytotable_peak_memory (multithread) (GB) (mean)",
        "pycytominer_peak_memory (GB) (mean)",
    ],
    title="CytoTable and Pycytominer SQLite Peak Memory with Min/Max Errors",
    labels={"data_input_renamed": "Input File", "value": "Gigabytes (GB)"},
    height=500,
    width=900,
    symbol_sequence=["diamond"],
    color_discrete_sequence=[
        px.colors.qualitative.Vivid[6],
        px.colors.qualitative.Vivid[7],
        px.colors.qualitative.Vivid[4],
    ],
)

# Add error bars for each trace
for i, col in enumerate(
    [
        "cytotable_peak_memory (multiprocess) (GB)",
        "cytotable_peak_memory (multithread) (GB)",
        "pycytominer_peak_memory (GB)",
    ]
):
    fig.data[i].update(
        error_y=dict(
            array=(
                aggregated_results[f"{col} (max)"] - aggregated_results[f"{col} (mean)"]
            ),
            arrayminus=(
                aggregated_results[f"{col} (mean)"] - aggregated_results[f"{col} (min)"]
            ),
        )
    )

# Rename the lines for the legend
newnames = {
    "cytotable_peak_memory (multiprocess) (GB) (mean)": "CytoTable (multiprocess)",
    "cytotable_peak_memory (multithread) (GB) (mean)": "CytoTable (multithread)",
    "pycytominer_peak_memory (GB) (mean)": "Pycytominer",
}
fig.for_each_trace(
    lambda t: t.update(
        name=newnames[t.name],
        legendgroup=newnames[t.name],
        hovertemplate=t.hovertemplate.replace(t.name, newnames[t.name]),
    )
)

# Update the legend and layout
fig.update_layout(
    legend_title_text="",
    legend=dict(x=0.01, y=0.98, bgcolor="rgba(255,255,255,0.8)"),
    font=dict(size=16),  # Global font size
)
fig.update_traces(mode="lines+markers")
fig.update_layout(yaxis=dict(rangemode="tozero", autorange=True))

# Save and display the plot
fig.write_image(join_mem_size_image)
fig.write_image(join_mem_size_image.replace(".png", ".svg"))
Image(url=join_mem_size_image.replace(".png", ".svg"))